## Este codigo genera los hologramas de amplitud-fase de modos Lagure-Gauss, con una interfaz que permite controlar parametros del haz y activar superposicion de haces


In [17]:
import cv2
import screeninfo
import numpy as np


Nxpix = 1024 ## DIMENSIONES DEL SLM
Nypix = 768
Pix_size=9e-6 #TAMANO DEL PIXEL ES 9um




from scipy.special import genlaguerre

def laguerre_gauss_mode(p, l, r, phi, z, w0, k):
    """
    Computes the Laguerre-Gaussian mode of light.
    
    Parameters:
        p  : Radial index (non-negative integer)
        l  : Azimuthal index (integer)
        r  : Radial coordinate
        phi: Azimuthal coordinate
        z  : Propagation distance
        w0 : Beam waist
        k  : Wave number
    
    Returns:
        Complex field amplitude of the LG mode.
    """
    z_R = np.pi * w0**2 / k  # Rayleigh range
    w_z = w0 * np.sqrt(1 + (z / z_R) ** 2)  # Beam waist at z
    R_z = z * (1 + (z_R / z) ** 2) if z != 0 else np.inf  # Radius of curvature
    psi_z = np.arctan(z / z_R)  # Gouy phase
    
    C_lp = np.sqrt(2 * np.math.factorial(p) / (np.pi * np.math.factorial(p + abs(l))))
    R = np.sqrt(2) * r / w_z
    L = genlaguerre(p, abs(l))(2 * r**2 / w_z**2)  # Generalized Laguerre polynomial
    
    amplitude = C_lp * (R**abs(l)) * np.exp(-r**2 / w_z**2) * L
    phase = np.exp(-1j * k * r**2 / (2 * R_z)) * np.exp(-1j * l * phi) * np.exp(1j * psi_z)
    
    return (1 / w_z) * amplitude * phase

# Define grid
N = (Nxpix,Nypix)  # Grid size
x = np.linspace(-Nxpix/2*Pix_size, Nxpix/2*Pix_size, Nxpix)
y = np.linspace(-Nypix/2*Pix_size, Nypix/2*Pix_size, Nypix)
offsetx=int(input("Offset en x en pixeles: "))
offsety=int(input("Offset en y en pixeles: "))
X, Y = np.meshgrid(x-offsetx*Pix_size, y-offsety*Pix_size)
R = np.sqrt(X**2 + Y**2)
Phi = np.arctan2(Y, X)





                              
    
    
    
    ## FUNCION PARA MEJORAR EFICIENCIA DE SLM QUE NO SE MODULAN HASTA 2PI, 
                                ##PARA NUESTRO SLM C OPTIMO ES 2.1, C=1 no hace nada
                                ## "Optimisation of a low cost SLM for diffraction
                                ## efficiency and ghost order suppression"" (R.Bowman)

def contrast_function(C, array):
    y = C * (array - 255/2) + 255/2
    return np.clip(y, 0, 255).astype("uint8")


##GENERACION DE PATRONES AMPLITUD-FASE SEGUN (E.Bolduc,2013) "Exact solution to simultaneous intensity and phase encryption with a single phase-only hologram"

def Amplitude_Phase_pattern(Desired_Field, LAMBDA, NX, NY, Contrast=1):
    A = np.abs(Desired_Field)
    PHI = np.angle(Desired_Field)

    M = A / np.max(A)
    F = PHI - np.pi * M

    # Eliminamos los bucles for usando operaciones vectorizadas
    x = np.arange(NX)
    y = np.arange(NY)
    X, Y = np.meshgrid(x, y, indexing='xy')

    psi = M * np.mod(F + (2 * np.pi * X / LAMBDA) + (2 * np.pi * Y / LAMBDA), 2 * np.pi)

    psi_int = ((psi - psi.min()) * (255 / (psi.max() - psi.min()))).astype(np.uint8)

    return contrast_function(Contrast, psi_int)











# Función para mostrar la imagen en pantalla completa
def show_fullscreen_image(image, fullscreen=True, screen_index=0):
    if image is None:
        print("Error: Unable to load image.")
        return
    
    screens = screeninfo.get_monitors()
    if screen_index >= len(screens):
        print("Error: Screen index out of range.")
        return
    
    screen = screens[screen_index]
    screen_width, screen_height = screen.width, screen.height
    
    image = cv2.resize(image, (screen_width, screen_height))
    
    cv2.namedWindow("SLM Pattern", cv2.WND_PROP_FULLSCREEN)
    cv2.namedWindow("Control Panel", cv2.WINDOW_NORMAL)  # Asegura que la ventana existe
    cv2.moveWindow("SLM Pattern", screen.x, screen.y)

    if fullscreen:
        cv2.setWindowProperty("SLM Pattern", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    cv2.imshow("SLM Pattern", image)
    

# Función para actualizar la imagen en tiempo real
trackbars_ready = False

def update_image(_):
    global l, p, waist, contrast
    if not trackbars_ready:
        return  # No hacer nada hasta que los trackbars estén listos
    
    
    l = cv2.getTrackbarPos("l", "Control Panel")
    p = cv2.getTrackbarPos("p", "Control Panel")
    waist = max(cv2.getTrackbarPos("Waist (um)", "Control Panel") * 1e-6, 1e-6)
    contrast = cv2.getTrackbarPos("Contrast", "Control Panel")

    # Aquí generas la imagen sin riesgo de error
    try:
        
            
        Output_Field = laguerre_gauss_mode(p, l, R, Phi, z, waist, k)
        
            
          
        if superposition==True:
            
            Output_Field += laguerre_gauss_mode(p_, l_, R, Phi, z, waist, k)
           
        
        
         
        Hologram = Amplitude_Phase_pattern(Output_Field, Pix_size, Nxpix, Nypix, Contrast=contrast)
        show_fullscreen_image(Hologram, fullscreen=fullscreen_mode,screen_index=0)
        
    except ZeroDivisionError:
        print("⚠️ División por cero evitada: w0 o z_R demasiado pequeños. Ajusta los parámetros.")




# Parámetros iniciales
p, l = 0, 4
waist = 1000e-6# Tamaño del haz inicial en metros
z = 0  # Propagation distance
k = 2*np.pi/633e-9




contrast = 1
fullscreen_mode = False # Control de pantalla completa
superposition=False



def draw_instructions(img):
    instructions = ["Ingresar los valores numericos en consola",
        "Teclas de control:",
        "q - Salir",
        "f - Alternar pantalla completa",
        "s - Alternar superposicion de haces",
        "w - Ajustar waist manualmente",
        "c - Ajustar contraste manualmente",
        "+ - Aumentar waist (50um)",
        "- - Disminuir waist(50um)",
    ]
    y0, dy = 20, 20  # Posición inicial y espaciado
    for i, line in enumerate(instructions):
        y = y0 + i * dy
        cv2.putText(img, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
    return img

# Crear ventana de instrucciones
cv2.namedWindow("Instrucciones", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Instrucciones", 400, 400)

# Crear imagen de fondo para el panel
panel_img = np.zeros((400,400 , 3), dtype=np.uint8)
panel_img = draw_instructions(panel_img)

cv2.imshow("Instrucciones", panel_img)



# Crear la ventana del panel de control
cv2.namedWindow("Control Panel", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Control Panel", 400, 400)  # Tamaño ajustable

# Imagen de fondo para el panel

# Crear sliders
cv2.createTrackbar("l", "Control Panel", l, 10, update_image)
cv2.createTrackbar("p", "Control Panel", p, 10, update_image)
cv2.createTrackbar("Waist (um)", "Control Panel", int(waist * 1e6), 2000, update_image)
cv2.createTrackbar("Contrast", "Control Panel", int(contrast), 255, update_image)

cv2.waitKey(50)

trackbars_ready = True

# Generar primera imagen
update_image(0)

# Esperar hasta que el usuario presione 'q' para salir o 'f' para pantalla completa
while True:
    
    
    
   
    key = cv2.waitKey(1) & 0xFF
  

    if key == ord('q'):  # Salir con 'q'
        break
        
    elif key == ord('+'):
        
        waist +=50e-6
     
        cv2.setTrackbarPos("Waist (um)", "Control Panel", int(waist * 1e6))
        
        update_image(0)
    elif key == ord('-'):
        waist -= 50e-6
        
        cv2.setTrackbarPos("Waist (um)", "Control Panel", int(waist * 1e6))
        update_image(0)

    elif key == ord('f'):  # Alternar entre pantalla completa y ventana normal
        fullscreen_mode = not fullscreen_mode
        update_image(0)
        
    elif key == ord('s'):  # Alternar entre pantalla completa y ventana normal
        superposition =not superposition
        
        if superposition:
            p_=int(input("Ingresar p del segundo haz: "))
            l_=int(input("Ingresar l del segundo haz: "))
            
        
        update_image(0)
        
        if superposition:
            print("superposicion activada")
        
        else :
            print("superposicion desactivada")
        

        
    elif key == ord('w'):  # Ingresar waist manualmente
        try:
            new_waist = float(input("Ingrese el nuevo tamaño de waist (um): ")) * 1e-6
            if 1e-6 <= new_waist <= 2000e-6:
                waist = new_waist
                cv2.setTrackbarPos("Waist (um)", "Control Panel", int(waist * 1e6))
                update_image(0)
            else:
                print("⚠️ Valor fuera de rango (1-2000 um)")
        except ValueError:
            print("⚠️ Entrada no válida.")
    elif key == ord('c'):  # Ingresar contraste manualmente
        try:
            new_contrast = int(input("Ingrese el nuevo contraste (1-255): "))
            if 1 <= new_contrast <= 255:
                contrast = new_contrast
                cv2.setTrackbarPos("Contrast", "Control Panel", contrast)
                update_image(0)
            else:
                print("⚠️ Valor fuera de rango (1-255)")
        except ValueError:
            print("⚠️ Entrada no válida.")

cv2.destroyAllWindows()


Offset en x en pixeles: 0
Offset en y en pixeles: 0
Ingresar p del segundo haz: 3
Ingresar l del segundo haz: -3
superposicion activada
